In [1]:
from text_processing import *
from sklearn.metrics import precision_recall_fscore_support
from constants import *
from functools import reduce

In [2]:
file_selector = is_2017_in_file

In [3]:
judgements_files = filter(file_selector, os.listdir(json_data_dir))
texts = chain.from_iterable(map(lambda f: judgements_raw(f,json_data_dir), judgements_files))
texts = islice(texts,2000)
filtered = filter(filter_judgements, texts)

In [4]:
with_categories = map(lambda j: WithCategory(j, map_category(j).label), filtered)
cleaned = map(
    lambda j: Line(j.judgement['id'], clean_text(j.judgement['textContent'], common_words), j.category),
    with_categories)
cleaned  = list(cleaned)

### Nr of words in raw dataset

In [6]:
reduce(lambda x,y: x+y, map(lambda l: len(l.words.split()),cleaned))

3686640

In [7]:
tuples = map(lambda l: (l.words,l.category),cleaned)
listed = list(filter(lambda tup: tup[1] in ['civil', 'criminal', 'economic', 'insurance'],tuples))

x_train, x_test, y_train, y_test = get_train_test_data(listed)



In [8]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words = common_words)),
    ('clf', OneVsRestClassifier(LinearSVC())),
])

pipeline.fit(x_train, y_train)
predictions = pipeline.predict(x_test)

print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.90      0.97      0.93       167
          1       1.00      0.99      0.99        79
          2       0.75      0.10      0.18        30
          3       1.00      0.99      0.99        78

avg / total       0.93      0.90      0.89       354



In [9]:
precision_recall_fscore_support(y_test, predictions,average= 'micro')

(0.9411764705882353, 0.903954802259887, 0.9221902017291066, None)

In [10]:
precision_recall_fscore_support(y_test, predictions,average= 'macro')

(0.9112569060773481, 0.7611452848927267, 0.7736710078347743, None)

In [11]:
cat_id = dict(map(lambda line: (line.id, line.category), cleaned))
tagged_categories = list(tagged(cat_id))

In [13]:
tuples = map(lambda l: (l.words,l.category),tagged_categories)
listed = list(filter(lambda tup: tup[1] in ['civil', 'criminal', 'economic', 'insurance'],tuples))

('sygnatura:brev akt:subst ka:subst wyrok:subst imię:subst rzeczpospolita:subst polski:adj maj:subst okręgowy:adj toruń:subst skład:subst przewodniczący:subst s:brev s:brev o:brev aleksandra:subst nowicka:subst sędzia:subst s:brev s:brev o:brev barbar:subst plewińska:subst s:brev s:brev o:brev rafał:subst sadowski:subst spr:subst protokolant:subst stopień:brev sekr:brev katarzyna:subst kotarska:subst przy:prep udział:subst prokurator:subst prok:subst rejon:subst toruń:subst centrum:subst zachód:subst marcin:subst reszkowskiego:subst po:prep rozpoznać:ger maj:subst sprawa:subst r:brev oskarżyć:ppas kk:brev zw:brev kk:brev oraz:conj kk:brev zw:brev kk:brev skutek:subst apelacja:subst wnieść:ppas oskarżony:subst oraz:conj obrońca:subst wyrok:subst rejonowy:adj toruń:subst październik:subst sygnatura:brev akt:subst k:subst zaskarżyć:ppas wyrok:subst całość:subst utrzymywać:fin moc:subst zasądzać:fin skarb:subst państwo:subst rejonowy:adj toruń:subst rzecz:subst kancelaria:subst adwokacki:a

In [14]:
x_train, x_test, y_train, y_test = get_train_test_data(listed)

In [15]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words = common_words)),
    ('clf', OneVsRestClassifier(LinearSVC())),
])

pipeline.fit(x_train, y_train)
predictions = pipeline.predict(x_test)

print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.86      0.97      0.91       167
          1       1.00      0.95      0.97        79
          2       1.00      0.03      0.06        30
          3       1.00      0.94      0.97        78

avg / total       0.93      0.88      0.87       354



In [16]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words = common_tagged_words)),
    ('clf', OneVsRestClassifier(LinearSVC())),
])

pipeline.fit(x_train, y_train)
predictions = pipeline.predict(x_test)

print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.86      0.97      0.91       167
          1       1.00      0.95      0.97        79
          2       1.00      0.03      0.06        30
          3       1.00      0.94      0.97        78

avg / total       0.93      0.88      0.87       354



In [17]:
precision_recall_fscore_support(y_test, predictions,average= 'micro')

(0.9201183431952663, 0.8785310734463276, 0.8988439306358381, None)

In [18]:
precision_recall_fscore_support(y_test, predictions,average= 'macro')

(0.9642857142857143, 0.7221644345194713, 0.7288854699568342, None)